# Style Transport
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mdehling/kolkin-style-transport/blob/main/style-transport.ipynb)

This notebook lets you try out image stylization using the approach described
by Kolkin, Salavon, and Shakhnarovich in their article _Style Transfer by
Relaxed Optimal Transport and Self-Similarity_.  For more details, see my
github repositories [`kolkin-style-transport`](
https://github.com/mdehling/kolkin-style-transport) and
[`nstesia`](https://github.com/mdehling/nstesia).

In [ ]:
# At the time of writing, the versions of pip and setuptools provided by colab
# do not have full `pyproject.toml` support --- they must be updated before
# installing the nstesia package.  This cell will do just that.
try:
    from google import colab

    # Pull everything else from the repository in to the Colab environment.
    !git config --global init.defaultBranch main
    !git init .
    !git remote add origin https://github.com/mdehling/kolkin-style-transport.git
    !git pull --depth=1 origin main

    # These are just to avoid some scary-looking (but harmless) error messages.
    !pip uninstall -q -y numba
    !pip install -q 'jedi>=0.10'

    # Minimum versions required for PEP-660 support.
    !pip install -q 'pip>=21.3' 'setuptools>=64'

    # Finally install the one we came for.
    !pip install -q -r requirements-colab.txt

except ImportError:
    pass

In [ ]:
from IPython.display import display

from os import environ as env
env['TF_CPP_MIN_LOG_LEVEL'] = '2'               # hide info & warnings
env['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'       # grow GPU memory as needed

import tensorflow as tf
import nstesia as nst

In [ ]:
# When using Colab, select 'Change runtime type' from the 'Runtime' menu.
assert tf.config.list_physical_devices('GPU'), \
    "you really shouldn't run this notebook without a gpu"


## Simple Image Stylization

In [ ]:
def show_image(image_tensor):
    display(tf.keras.utils.array_to_img(tf.squeeze(image_tensor,axis=0)))

In [ ]:
content_image = nst.io.load_image('img/content/brad.jpg')
style_image = nst.io.load_image('img/style/van-gogh-self-portrait.jpg')

pastiche_image = nst.kolkin_2019.generate_pastiche_image(
    content_image, style_image,
    content_weight=768.0,
    learning_rate=3.0,
    iterations=200,
)
show_image(pastiche_image)

In [ ]:
# img/results/karya-to-flowers.png
content_image = nst.io.load_image('img/content/karya.jpg')
style_image = nst.io.load_image('img/style/bruegel-flowers.jpg')

content_weights = [768, 384, 192, 96]
N = len(content_weights)

pastiche_images = []
for content_weight in content_weights:
    pastiche_image = nst.kolkin_2019.generate_pastiche_image(
        content_image, style_image,
        content_weight=content_weight,
        learning_rate=1.0,
        iterations=600,
    )
    pastiche_images.append(pastiche_image)

grid_image = nst.image.grid(
    [content_image] + [None]*(N-2) + [style_image] + pastiche_images,
    ncols=N,
)
show_image(grid_image)

## Content-Style Grids

In [ ]:
def content_style_grid(
    content_images,
    style_images,
    learning_rate=1.0,
    content_weight=512.0,
    iterations=600,
):
    """
    Show a grid of stylizations with content images as rows, styles as columns.

    Args:
        content_images:
            A list of image tensors representing the content images.
        style_images:
            A list of image tensors representing the style images.

    Returns:
        An image representing the grid of stylizations.
    """
    images = [None] + style_images
    for content_image in content_images:
        images += [content_image] + [
            nst.kolkin_2019.generate_pastiche_image(
                content_image, style_image,
                learning_rate=learning_rate,
                content_weight=content_weight,
                iterations=iterations,
            ) 
            for style_image in style_images
        ]

    return nst.image.grid(images, ncols=len(style_images)+1)

In [ ]:
# img/results/content-style-matrix-1.png
content_images = [
    nst.io.load_image('img/content/sunflower.jpg'),
    nst.io.load_image('img/content/bochum.jpg'),
]
style_images = [
    nst.io.load_image('img/style/wescoat-candy.jpg'),
    nst.io.load_image('img/style/picasso-briqueterie-a-tortosa.jpg'),
    nst.io.load_image('img/style/picabia-udnie.jpg'),
]
grid_image = content_style_grid(content_images, style_images, content_weight=768)
show_image(grid_image)

In [ ]:
# img/results/content-style-matrix-2.png
content_images = [
    nst.io.load_image('img/content/brad.jpg'),
    nst.io.load_image('img/content/karya.jpg'),
]
style_images = [
    nst.io.load_image('img/style/petitjean-femmes-au-bain.jpg'),
    nst.io.load_image('img/style/delaunay-portrait-de-metzinger.jpg'),
    nst.io.load_image('img/style/munch-skrik.jpg'),
]
grid_image = content_style_grid(content_images, style_images, content_weight=512)
show_image(grid_image)